<a href="https://colab.research.google.com/github/tritom88/HEROKU-RCLONE-SERVE-INDEX/blob/main/Qbitt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 📚 For more information please visit our [GitHub](https://github.com/MinorMole/RcloneLab/).


# Mount with Google Drive module
#### (NOT RECOMMENDED, SOMETIMES WRITING ISSUES)

In [ ]:
#@ Check enabled if you want to run this cell.
enabled = False #@param {type:"boolean"}

if enabled:
  # Run this cell to mount your Google Drive.
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Not enabled. Skipping....")

#Mount with Rclone 
#### (RECOMMENDED)

In [ ]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "uploaddrive" #@param {type:"string"}
local_mount_location = "/content/udrive/" #@param {type:"string"}
#@markdown The default qBittorrent download location is '/content/udrive/' (they get downloaded to '/content/qbittorrent/' first)
# ============================= FORM ============================= #
import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M & 
  


# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

# <img src='https://szyha.github.io/RcloneLabArchive/img/title_qbittorrent.png' height="45" alt="qBittorrent"/>

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝CREATE CONFIG FILE</h3>
connections = 450 #@param {type:"integer"}
savePath = "/content/udrive/" #@param {type:"string"}
install_search_plugins = True #@param {type:'boolean'}
#https://github.com/thim0o/Google-Colab-Peerflix-Server
# ============================= FORM ============================= #

import os
import logging
import json
import requests
import ast

configPath = r"/root/.config/qBittorrent/qBittorrent.conf"


#CREATE THE CONFIG
data = r"""
[AutoRun][AutoRun]
enabled=true
program=chmod -R 666 \"%D\"

[BitTorrent]
Session\CreateTorrentSubfolder=true
Session\DisableAutoTMMByDefault=false
Session\DisableAutoTMMTriggers\CategoryChanged=false
Session\DisableAutoTMMTriggers\CategorySavePathChanged=false
Session\DisableAutoTMMTriggers\DefaultSavePathChanged=true
Session\GlobalMaxSeedingMinutes=1000

[Core]
AutoDeleteAddedTorrentFile=IfAdded

[Preferences]
Downloads\DiskWriteCacheSize=64
Session\SuggestMode=true
Advanced\AnnounceToAllTrackers=true
Session\AnnounceToAllTiers=true
Bittorrent\AddTrackers=true
Bittorrent\MaxConnecs={connections}
Bittorrent\MaxConnecsPerTorrent=-1
Bittorrent\MaxRatio=4
Bittorrent\MaxRatioAction=0
Bittorrent\MaxUploads=8
Bittorrent\PeX=true
Bittorrent\TrackersList=http://nyaa.tracker.wf:7777/announce\n\nhttp://sukebei.tracker.wf:8888/announce\n\nudp://tracker.coppersurfer.tk:6969/announce\n\nudp://tracker.open-internet.nl:6969/announce\n\nudp://tracker.leechers-paradise.org:6969/announce\n\nudp://tracker.internetwarriors.net:1337/announce\n\nudp://tracker.opentrackr.org:1337/announce\n\nudp://9.rarbg.to:2710/announce\n\nudp://9.rarbg.me:2710/announce\n\nhttp://tracker3.itzmx.com:6961/announce\n\nhttp://tracker1.itzmx.com:8080/announce\n\nudp://exodus.desync.com:6969/announce\n\nudp://explodie.org:6969/announce\n\nudp://ipv4.tracker.harry.lu:80/announce\n\nudp://denis.stalker.upeer.me:6969/announce\n\nudp://tracker.torrent.eu.org:451/announce\n\nudp://tracker.tiny-vps.com:6969/announce\n\nudp://thetracker.org:80/announce\n\nudp://open.demonii.si:1337/announce\n\nudp://tracker4.itzmx.com:2710/announce\n\nudp://tracker.cyberia.is:6969/announce\n\nudp://retracker.netbynet.ru:2710/announce\n\nudp://62.138.0.158:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://188.241.58.209:6969/announce\n\nudp://185.225.17.100:1337/announce\n\nudp://62.210.79.110:1337/announce\n\nudp://151.80.120.112:2710/announce\n\nudp://151.80.120.114:2710/announce\n\nhttp://176.113.71.19:6961/announce\n\nhttp://172.64.195.37:8080/announce\n\nudp://208.83.20.20:6969/announce\n\nudp://184.105.151.164:6969/announce\n\nudp://51.15.40.114:80/announce\n\nudp://5.2.79.22:6969/announce\n\nudp://91.216.110.52:451/announce\n\nudp://5.206.58.23:6969/announce\n\nudp://176.31.106.35:80/announce\n\nudp://5.2.79.219:1337/announce\n\nudp://212.1.226.176:2710/announce\n\nudp://159.100.245.181:6969/announce\n\nudp://212.1.226.176:2710/announce\n
Bittorrent\uTP_rate_limited=false
Connection\GlobalDLLimitAlt=0
Connection\GlobalUPLimit=30000
Connection\GlobalUPLimitAlt=0
Connection\PortRangeMin=12121
Downloads\PreAllocation=false
Downloads\SavePath={savePath}
Downloads\ScanDirsV2=@Variant(\0\0\0\x1c\0\0\0\x1\0\0\0\x12\0/\0\x63\0o\0n\0t\0\x65\0n\0t\0/\0\0\0\x2\0\0\0\x1)
Downloads\StartInPause=false
Downloads\TempPath=/content/qBittorrent/
Downloads\TempPathEnabled=true
General\Locale=en
General\UseRandomPort=false
Queueing\IgnoreSlowTorrents=true
Queueing\MaxActiveDownloads=8
Queueing\MaxActiveTorrents=15
Queueing\MaxActiveUploads=8
Queueing\QueueingEnabled=true
WebUI\Address=*
WebUI\AlternativeUIEnabled=false
WebUI\AuthSubnetWhitelist=@Invalid()
WebUI\AuthSubnetWhitelistEnabled=false
WebUI\ClickjackingProtection=true
WebUI\HTTPS\Enabled=false
WebUI\HostHeaderValidation=true
WebUI\LocalHostAuth=false
WebUI\Password_ha1=@ByteArray(322517f6a6fc65697957a1f9da700b43)
WebUI\Port=6006
WebUI\RootFolder=
WebUI\Username=rclonelab
""".format(connections=connections,
          savePath=savePath,
          ).split("\n")




# SAVE THE CONFIG
!mkdir /root/.config/qBittorrent/
open(configPath, 'w').close()

configFile = open(configPath, "a+")

for line in data:
  configFile.write(line + "\n")
  
configFile.close()


# PRINT THE CONFIG FOR DEBUGGING
f = open(configPath, "r")
text = f.read()
f.close()

clear_output(wait=True)
print("created the config successfully")

f = open(r"nohup.out", "w+").close()
# Download search engines

engineDirectory = r"/root/.local/share/data/qBittorrent/nova3/engines/"
if install_search_plugins:
  
  !mkdir $engineDirectory >/dev/null
  pluginsInstalled = 0
  searchEngineLinks = [r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/kickass_torrent.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/extratorrent.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/ettv.py",
                       r"https://raw.githubusercontent.com/hannsen/qbittorrent_search_plugins/master/demonoid.py",
                       r"https://raw.githubusercontent.com/MaurizioRicci/qBittorrent_search_engine/master/torrentproject.py",
                       r"https://raw.githubusercontent.com/Pireo/hello-world/master/rutor.py",
                       r"https://raw.githubusercontent.com/nindogo/qbtSearchScripts/master/magnetdl.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/zooqle.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/rarbg.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/torlock.py",
                       r"https://github.com/qbittorrent/search-plugins/blob/master/nova3/engines/btdb.py",
                       
                        ]
  cmd = ""
  for link in searchEngineLinks:
    !nohup wget -nc -P /root/.local/share/data/qBittorrent/nova3/engines/ $link >/dev/null 2>&1 &
    pluginsInstalled += 1
    clear_output(wait=True)
  !$cmd

  print(pluginsInstalled, "plugins installed")

f = open(r"nohup.out", "r")
nohupText = f.read()
print(nohupText)

dirs = os.listdir(engineDirectory)
print(len(dirs), "in plugins folder")

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 INSTALL QBITTORRENT AND FILE MANAGER</h3>
Old_Version = False #@param {type:"boolean"}
Install_File_Manager = True #@param {type:"boolean"}
subdomain = "random" #@param {type:"string"}


# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output

#kill qbittorrent to be able to reload config
!pkill qbittorrent-nox
!pkill autossh
SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue
  
def AutoSSH(name,port):
  get_ipython().system_raw("autossh -l " + name + " -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:" + port + " ssh.localhost.run &")
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_Server():
  if CheckProcess("qbittorrent-nox", "") != True:
    print("Starting qbittorrent...")
    !qbittorrent-nox -d --webui-port=6006
  if CheckProcess("autossh", Random_Number) != True:
      AutoSSH(Random_Number, "6006")
  if Install_File_Manager == True:
    if os.path.isfile("/tools/node/bin/cloudcmd") == False:
      clear_output(wait=True)
      !npm i -g npm
      !npm i cloudcmd -g
    if CheckProcess("cloudcmd", "") != True:
      get_ipython().system_raw("cloudcmd --online --no-auth --show-config --show-file-name --editor 'deepword' --packer 'tar' --port 7007 --progress --no-confirm-copy --confirm-move --name 'RcloneLab File Manager' --keys-panel --no-contact --console --sync-console-path --no-terminal --no-vim --columns 'name-size-date' --no-log &")
    if CheckProcess("autossh", "fm" + Random_Number) != True:
      AutoSSH("fm" + Random_Number, "7007")



try:
  if subdomain == "random":
    
    Random_NumberMT = RandomGenerator()
    Random_Number = RandomGenerator()
  else:
    Random_NumberMT = subdomain
    Random_Number = subdomain

#    !rm -rf /content/{sample_data,.config}/
    
  if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    !apt update -qq -y
    !yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable
    if Old_Version == True:
      !apt install qbittorrent-nox=4.0.3-1 -qq -y
    else:
      !apt install qbittorrent-nox -qq -y
      
    clear_output(wait=True)

  if os.path.isfile("/usr/bin/autossh") == False:
    !apt install autossh -qq -y
  Start_Server()
  display(SuccessRun)
  display(MakeButton("Recheck", Start_Server))
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">qBittorrent</h2><h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"http://" + Random_Number + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
               "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"http://" + Random_Number + ".serveo.net\" target=\"_blank\">Website 2</a></h4>"))
  if Install_File_Manager == True:
        display(HTML(
            "<h2 style=\"font-family:Trebuchet MS;color:#446785;\">File Manager</h2><h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://fm" + Random_Number + ".localhost.run/fs/content/\" target=\"_blank\">Website 1</a><br>"  "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://fm" + Random_Number + ".serveo.net/fs/content/\" target=\"_blank\">Website 2</a></h3>"))
except Exception as e:
  console.log(e)
  display(UnsuccessfullyRun)

# <img src='https://szyha.github.io/RcloneLabArchive/img/title_jdownloader.png' height="45" alt="JDownloader"/>

In [ ]:
#@markdown <h3>⬅️ Click Here to Install JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Text(placeholder="/content/Downloads", description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML(""))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-8-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()

# <img src='https://szyha.github.io/RcloneLabArchive/img/title_youtube-dl.png' height="45" alt="Youtube-DL"/>

In [ ]:
#@markdown <h3>⬅️ Click Here to Use YouTube-DL (Beta)</h3>

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from google.colab import output

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"], description="Video:")

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"], description="Audio:")

Subtitle = widgets.Dropdown(options=["Enable", "Disable"], description="Subtitle:")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"], description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def youtube_id(value):
    query = urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    return None

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(Links, VideoQ, AudioQ, Subtitle, SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML(""))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Video
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      os.environ["videoqC"] = "webm"
    else:
      os.environ["videoqC"] = "mp4"
    # Audio
    if AudioQ.value == "Best Quality (Opus)":
      os.environ["audioqC"] = "webm"
    else:
      os.environ["audioqC"] = "m4a"
    # Subtitle
    if Subtitle.value == "Enable":
      os.environ["subtitleC"] = "--all-subs --convert-subs ass --embed-subs"
    else:
      os.environ["subtitleC"] = ""
    # Compatibility Extension
    if Subtitle.value == "Disable" and AudioQ.value == "Best Compatibility (M4A)":
      os.environ["extC"] = "mp4"
      os.environ["thumbnailC"] = "--embed-thumbnail"
    else:
      os.environ["extC"] = "mkv"
      os.environ["thumbnailC"] = ""
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing " + str(Count) + " out of " + Total + "</h3>"))
      if youtube_id(Link) != None:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Video currently downloading...</h3><br>"), YouTubeVideo(youtube_id(Link), width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading " + Link + "</h3><br>"))
      os.environ["Link"] = Link
      !youtube-dl -i --no-warnings --geo-bypass --yes-playlist --add-metadata --merge-output-format $extC $thumbnailC $subtitleC -f "bestvideo[ext=$videoqC]+bestaudio[ext=$audioqC]/bestvideo+bestaudio" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
ShowYT()

# <img src='https://szyha.github.io/RcloneLabArchive/img/title_rclonelab.png' height="45" alt="RcloneLab"/>

In [ ]:
# ============================= FORM ============================= #
Mode = "Off" #@param ["Off","Config", "Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "" #@param {type:"string"}
Destination = "uploaddrive:/test/" #@param {type:"string"}
Transfers = 4 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 20 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---
 
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Simple_Ouput = True #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = True #@param {type:"boolean"}
Skip_all_files_that_exist = True #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False
Google_Drive_optimization = True #@param {type:"boolean"}
Dry_Run = False #@param {type:"boolean"}
Output_Log_File = "DEBUG" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = "" #@param {type:"string"}
 
#@markdown <center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}
 
#@markdown <center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}
# ================================================================ #
 
import os; from IPython.display import HTML, clear_output
import sys
 
 
if Mode != "Off":
 
  # Optimized for Google Colaboratory
  os.environ["bufferC"] = "--buffer-size 96M"
 
  if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
  elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
  elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
  elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
  else:
    os.environ["compareC"] = ""
 
  os.environ["sourceC"] = Source
  os.environ["destinationC"] = Destination
  os.environ["transfersC"] = "--transfers "+str(Transfers)
  os.environ["checkersC"] = "--checkers "+str(Checkers)
 
  if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
  else:
    os.environ["skipnewC"] = ""
 
  if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
  else:
    os.environ["skipexistC"] = ""
 
  if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
  else:
    os.environ["nocrossfilesystemC"] = ""
 
  if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
  else:
    os.environ["noupdatemodtimeC"] = ""
 
  if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
  else:
    os.environ["filesoptimizeC"] = ""
 
  if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever --disable copy"
  else:
    os.environ["driveoptimizeC"] = ""
 
  if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
  else:
    os.environ["dryrunC"] = ""
 
  if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
  else:
    if Simple_Ouput == True:
      os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
      os.environ["statsC"] = "-v --stats=5s"
 
  if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
  elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
  elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
  else:
    os.environ["loglevelC"] = ""
 
  os.environ["extraC"] = Extra_Arguments
 
  if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
  elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
  elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"
 
  if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
  else:
    os.environ["trackrenamesC"] = ""
 
  if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
  elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
  elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
  elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
  elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
  elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
  elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"
 
  if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
  else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"
 
  ### rclone Execution
 
  if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))
 
 
  if Mode == "Config":
    !rclone --config=/root/.rclone.conf config
  elif Mode == "Copy":
    !rclone --config=/root/.rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Move":
    !rclone --config=/root/.rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Sync":
    !rclone --config=/root/.rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Checker":
    !rclone --config=/root/.rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Deduplicate":
    !rclone --config=/root/.rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
  elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "Empty Trash":
    !rclone --config=/root/.rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC
  elif Mode == "qBittorrent":
    !chmod -R 666 /content/qBittorrent/
    !rclone --config=/root/.rclone.conf move "/content/qBittorrent/" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs --exclude **/.unwanted/ $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
 
  ### Log Output
 
  if Output_Log_File != "OFF" and Mode != "Config":
    ### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
      f.write("Mode: " + Mode + \
              "\nCompare: " + Compare + \
              "\nSource: \"" + Source + \
              "\"\nDestination: \"" + Destination + \
              "\"\nTransfers: " + str(Transfers) + \
              "\nCheckers: " + str(Checkers) + \
              "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
              "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
              "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
              "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
              "\nDry-Run: " + str(Dry_Run) + \
              "\nOutput Log Level: " + Output_Log_File + \
              "\nExtra Arguments: \"" + Extra_Arguments + \
              "\"\nSync Moden: " + Sync_Mode + \
              "\nTrack Renames: " + str(Track_Renames) + \
              "\nDeduplicate Mode: " + Deduplicate_Mode + \
              "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
    ### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/
    ### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
      try:
        files.download("/root/rclone_log.zip")
        !rm -f /root/rclone_log.zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
      except:
        !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br><img src=\"https://minormole.github.io/RcloneLab/res/rclonelab/01.png\"><br></center>"))
    else:
      clear_output(wait=True)
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))
 
  ### Operation has been successfully completed.
  if os.path.isfile("/usr/bin/rclone") == False:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
  elif Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))
else:
  !echo "doing nothing because the mode is set to off"

# <img src='https://szyha.github.io/RcloneLabArchive/img/title_utility.png' height="45" alt="Utility"/>

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>Ubuntu Virtual Machine Updater</h3>
# ================================================================ #

from IPython.display import HTML, clear_output

!apt update -qq -y
!apt upgrade -qq -y
!npm i -g npm

clear_output(wait=True)
display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">An update has been successfully completed.</h2><br></center>"))

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>Check VM's Status</h3>
Loop_Check = False #@param {type:"boolean"}
Loop_Interval = 5 #@param {type:"slider", min:1, max:15, step:1}
# ================================================================ #

import time
from IPython.display import clear_output
Loop = True

try:
  while Loop == True:
    clear_output(wait=True)
    !top -bcn1 -w512
    if Loop_Check == False:
      Loop = False
    else:
      time.sleep(Loop_Interval)
except:
  clear_output(wait=True)

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>Get VM's Specification</h3>
Output_Format = "TEXT" #@param ["TEXT", "HTML", "XML", "JSON"]
Short_Output = False #@param {type:"boolean"}
# ================================================================ #

import os
from google.colab import files
from IPython.display import HTML, clear_output

try:
  Output_Format_Ext
except NameError:
  !apt install lshw -qq -y
  clear_output(wait=True)

if Short_Output == True:
  os.environ["outputformatC"] = "txt"
  os.environ["outputformat2C"] = "-short"
  Output_Format_Ext = "txt"
elif Output_Format == "TEXT":
  os.environ["outputformatC"] = "txt"
  os.environ["outputformat2C"] = ""
  Output_Format_Ext = "txt"
else:
  os.environ["outputformatC"] = Output_Format.lower()
  os.environ["outputformat2C"] = "-"+Output_Format.lower()
  Output_Format_Ext = Output_Format.lower()
  
!lshw $outputformat2C > Specification.$outputformatC
files.download("/content/Specification." + Output_Format_Ext)
!rm -f /content/Specification.$outputformatC
display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))

In [ ]:
# ============================= FORM ============================= #
#@markdown <br><center><img src='https://szyha.github.io/RcloneLabArchive/img/title_netdata.png' height="100" alt="netdata"/></center>
#@markdown <center><h3>Real-time Server Monitoring</h3></center><br>
# ============================= FORM ============================= #

TornOnNetdata = True #@param {type:"boolean"}

# ============================= FORM ============================= #
# ================================================================ #

import os, psutil, IPython, uuid, time
import ipywidgets as widgets

from IPython.display import HTML, clear_output
from google.colab import output
import sys




SuccessRun = widgets.Button(
    description='✔ Successfully',
    disabled=True,
    button_style='success'
)

UnsuccessfullyRun = widgets.Button(
    description='✘ Unsuccessfully',
    disabled=True,
    button_style='danger'
)

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def AutoSSH(name,port):
  get_ipython().system_raw("autossh -l " + name + " -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R 80:localhost:" + port + " ssh.localhost.run &")
  get_ipython().system_raw("autossh -M 0 -fNT -o 'StrictHostKeyChecking=no' -o 'ServerAliveInterval 300' -o 'ServerAliveCountMax 30' -R " + name + ":80:localhost:" + port + " serveo.net &")

def Start_ServerMT():
  if CheckProcess("netdata", "") != True:
    !/usr/sbin/netdata
  if CheckProcess("autossh", "mt" + Random_NumberMT) != True:
      AutoSSH("mt" + Random_NumberMT, "19999")

      
      
if TornOnNetdata:


  try:
    try:
      Random_NumberMT
    except NameError:
      Random_NumberMT = RandomGenerator()
    if os.path.isfile("/usr/bin/autossh") == False:
      !apt update -qq -y
      !apt install autossh -qq -y
      clear_output(wait=True)
    if os.path.isfile("/usr/sbin/netdata") == False:
      !bash <(curl -Ss https://my-netdata.io/kickstart.sh) --dont-wait --dont-start-it
      clear_output(wait=True)
    Start_ServerMT()
    display(SuccessRun)
    display(MakeButton("Recheck", Start_ServerMT))
    display(HTML("<h4 style=\"font-family:Trebuchet MS;color:#446785;\"><a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://mt" + Random_NumberMT + ".localhost.run\" target=\"_blank\">Website 1</a><br>" \
                 "<a style=\"font-family:Trebuchet MS;color:#356ebf;\" href=\"https://mt" + Random_NumberMT + ".serveo.net\" target=\"_blank\">Website 2</a></h4>"))
  except:
    clear_output(wait=True)
    display(UnsuccessfullyRun)
else:
  !echo "netdata not turned on"